In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)

def get_num_correct(preds, labels):
      return preds.argmax(dim=1).eq(labels).sum().item()
    

class Network(nn.Module):
      def __init__(self):
            super().__init__()
            self.conv1=nn.Conv2d(in_channels=1, out_channels=12, kernel_size=5)
            self.conv2=nn.Conv2d(in_channels=12, out_channels=24, kernel_size=5)

            self.fc1=nn.Linear(in_features=24*4*4, out_features=120)
            self.fc2=nn.Linear(in_features=120, out_features=60)
            self.out=nn.Linear(in_features=60, out_features=10)

      def forward(self, t):

            t=F.relu(self.conv1(t))
            t=F.max_pool2d(t, kernel_size=2, stride=2)

            t=F.relu(self.conv2(t))
            t=F.max_pool2d(t, kernel_size=2, stride=2)

            t=t.reshape(-1,24*4*4)
            t=F.relu(self.fc1(t))

            t=F.relu(self.fc2(t))

            t=self.out(t)

            return t

In [9]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
                                   transforms.ToTensor()
    ])
)

In [14]:
network = Network()#instance created

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)# batches formed
optimizer = optim.Adam(network.parameters(), lr=0.01)# values feeded to the parameter

for epoch in range(5):# running through 5 epochs

    total_loss = 0
    total_correct = 0 # to find how many correct predictions does our model make in each epoch seperately


    for batch in train_loader: #transversing through the train_loader which is the superset of our batch data
          images, labels = batch # Spliting up the batch

          preds = network(images)#feeding network with images
          loss = F.cross_entropy(preds, labels)# using cross_entropy method to calculate the loss

          optimizer.zero_grad()# setting gradiets back to zero so that they dont add up each time
          loss.backward()# starts the backpropogation algo
          optimizer.step() # updates the weights

          total_loss += loss.item()# sums up the losses
          total_correct += get_num_correct(preds, labels)# sums up the number of correct predinctions 

          
    print("epoch:", (epoch+1), "total correct:", total_correct, "loss:", total_loss)# prints the progress report of each epoch
print("Percentage Accuracy : ", total_correct/len(train_set))# tells us the accuracy of the model created after running through all the epochs

epoch: 1 total correct: 48418 loss: 311.9055641889572
epoch: 2 total correct: 51942 loss: 218.50904667377472
epoch: 3 total correct: 52466 loss: 204.573945119977
epoch: 4 total correct: 52775 loss: 195.31469969451427
epoch: 5 total correct: 53025 loss: 189.09671062231064
Percentage Accuracy :  0.88375
